## 4.3 듀얼 모멘텀 전략
* 상대 모멘텀
    * 투자 자산 가운데 상대적으로 상승 추세가 강한 종목에 투자
* 절대 모멘텀
    * 과거 시점 대비 현재 시점의 절대적 상승세를 평가
* 듀얼 모멘텀
    * 게리 안토나치 창시
    * 상대 모멘텀과 절대 모멘텀 결합
### 4.3.1 듀얼 모멘텀 전략 구현을 위한 절대 모멘텀 전략

In [1]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
# df = fdr.DataReader('US500', start='2000')
df = fdr.DataReader('spy', start='2000')
data = df.loc[:, ['Adj Close']].dropna().copy()
data.columns = ['Close']
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5911 entries, 2000-01-03 to 2023-06-30
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   5911 non-null   float64
dtypes: float64(1)
memory usage: 92.4 KB


In [3]:
# 말일 날짜 추출
data['STD_YM'] = data.apply(lambda x: x.name.strftime('%Y-%m'), axis=1)
month_last_df = data.drop_duplicates(['STD_YM'], keep="last").copy()
print(month_last_df.head())

                Close   STD_YM
Date                          
2000-01-31  90.773834  2000-01
2000-02-29  89.391708  2000-02
2000-03-31  98.055122  2000-03
2000-04-28  94.611313  2000-04
2000-05-31  93.123779  2000-05


In [4]:
# 1개월,12개월 전 값 추출
month_last_df['BF_1M_Close'] = month_last_df.shift(1)['Close']
month_last_df['BF_12M_Close'] = month_last_df.shift(12)['Close']
month_last_df.fillna(0, inplace=True)
month_last_df = month_last_df.loc['2008':]
print(month_last_df.head(15))

                 Close   STD_YM  BF_1M_Close  BF_12M_Close
Date                                                      
2008-01-31  101.562225  2008-01   108.097893    104.353462
2008-02-29   98.937599  2008-02   101.562225    102.306320
2008-03-31   98.052849  2008-03    98.937599    103.491920
2008-04-30  102.726280  2008-04    98.052849    108.076180
2008-05-30  104.279129  2008-05   102.726280    111.742149
2008-06-30   95.563911  2008-06   104.279129    110.108398
2008-07-31   94.705246  2008-07    95.563911    106.660851
2008-08-29   96.168755  2008-08    94.705246    108.029640
2008-09-30   87.112213  2008-09    96.168755    112.211983
2008-10-31   72.722435  2008-10    87.112213    113.734261
2008-11-28   67.660439  2008-11    72.722435    109.329071
2008-12-31   68.323288  2008-12    67.660439    108.097893
2009-01-30   62.712994  2009-01    68.323288    101.562225
2009-02-27   55.974525  2009-02    62.712994     98.937599
2009-03-31   60.637814  2009-03    55.974525     98.0528

In [5]:
# 포지션 기록
book = data['2008':].copy()
book['trade'] = ''
print(book.head())

                 Close   STD_YM trade
Date                                 
2008-01-02  107.151588  2008-01      
2008-01-03  107.099785  2008-01      
2008-01-04  104.475174  2008-01      
2008-01-07  104.386467  2008-01      
2008-01-08  102.700806  2008-01      


In [6]:
# 거래 실행, 매월 첫 영업일로 리밸런싱
ticker = 'S&P500'
for i in month_last_df.index:
    signal = ''
    momentum_index = month_last_df.loc[i,'BF_1M_Close'] / month_last_df.loc[i, 'BF_12M_Close'] - 1
    flag = True if ((momentum_index > 0.0) and (momentum_index != np.inf) and (momentum_index != -np.inf)) else False
    if flag:
        signal = 'buy ' + ticker
    print(f"날짜 : {i.strftime('%Y-%m-%d')}, 모멘텀 인덱스 : {momentum_index}, flag : {flag}, signal : {signal}")
    book.loc[i.strftime('%Y-%m'):, 'trade'] = signal # 매월 첫 영업일로 리밸런싱
print(book.loc[:'2008-02-01'])

날짜 : 2008-01-31, 모멘텀 인덱스 : 0.035882192389554035, flag : True, signal : buy S&P500
날짜 : 2008-02-29, 모멘텀 인덱스 : -0.0072732065819589575, flag : False, signal : 
날짜 : 2008-03-31, 모멘텀 인덱스 : -0.04400653693544376, flag : False, signal : 
날짜 : 2008-04-30, 모멘텀 인덱스 : -0.09274320206358144, flag : False, signal : 
날짜 : 2008-05-30, 모멘텀 인덱스 : -0.08068458572422832, flag : False, signal : 
날짜 : 2008-06-30, 모멘텀 인덱스 : -0.05294118437723516, flag : False, signal : 
날짜 : 2008-07-31, 모멘텀 인덱스 : -0.10403948492779225, flag : False, signal : 
날짜 : 2008-08-29, 모멘텀 인덱스 : -0.12334016849449836, flag : False, signal : 
날짜 : 2008-09-30, 모멘텀 인덱스 : -0.1429725023217886, flag : False, signal : 
날짜 : 2008-10-31, 모멘텀 인덱스 : -0.23407236980244683, flag : False, signal : 
날짜 : 2008-11-28, 모멘텀 인덱스 : -0.33482984594280507, flag : False, signal : 
날짜 : 2008-12-31, 모멘텀 인덱스 : -0.37408179639542094, flag : False, signal : 
날짜 : 2009-01-30, 모멘텀 인덱스 : -0.32727657354887596, flag : False, signal : 
날짜 : 2009-02-27, 모멘텀 인덱스 : -0.36613588126

In [7]:
# 전략 수익률
def returns(book, ticker):
    book['return'] = 1
    buy = 0.0
    signal = 'buy ' + ticker
    for i, x in enumerate(book.index):
        if book.loc[x, 'trade'] == signal and (i==0 or book.shift(1).loc[x, 'trade'] == ''):
            buy = book.loc[x, 'Close']
            print(f"진입일 : {x.strftime('%Y-%m-%d')}, 진입가격 : {buy}")
        elif buy != 0.0 and book.loc[x, 'trade'] == '' and book.shift(1).loc[x, 'trade'] == signal:
            sell = book.loc[x, 'Close']
            rtn = sell / buy
            print(f"청산일 : {x.strftime('%Y-%m-%d')}, 진입가격 : {buy}, 청산가격 : {sell}, return : {round(rtn, 4)}")
            buy = 0.0
        if book.shift(1).loc[x, 'trade'] == signal and i!=0:
            book.loc[x, 'return'] = book.loc[x, 'Close'] / book.shift(1).loc[x, 'Close']
    book['acc_ret'] = book['return'].cumprod()
    print(f"기간: {book.index[0].strftime('%Y/%m/%d')} ~ {book.index[-1].strftime('%Y/%m/%d')}")
    print(f"Accumulated return : {book.iloc[-1]['acc_ret']}")
    return round(book.iloc[-1]['acc_ret'], 4)

returns(book, ticker)
print(book.loc['2008':'2008-02-01'])

진입일 : 2008-01-02, 진입가격 : 107.151588
청산일 : 2008-02-01, 진입가격 : 107.151588, 청산가격 : 103.196152, return : 0.9631
진입일 : 2009-10-01, 진입가격 : 79.339203
청산일 : 2011-10-03, 진입가격 : 79.339203, 청산가격 : 88.092621, return : 1.1103
진입일 : 2011-11-01, 진입가격 : 97.764946
청산일 : 2012-01-03, 진입가격 : 97.764946, 청산가격 : 102.820381, return : 1.0517
진입일 : 2012-02-01, 진입가격 : 106.828377
청산일 : 2015-10-01, 진입가격 : 106.828377, 청산가격 : 167.135864, return : 1.5645
진입일 : 2015-11-02, 진입가격 : 183.020432
청산일 : 2016-02-01, 진입가격 : 183.020432, 청산가격 : 169.460739, return : 0.9259
진입일 : 2016-04-01, 진입가격 : 182.007019
청산일 : 2016-05-02, 진입가격 : 182.007019, 청산가격 : 182.930603, return : 1.0051
진입일 : 2016-06-01, 진입가격 : 184.95372
청산일 : 2019-01-02, 진입가격 : 184.95372, 청산가격 : 232.308746, return : 1.256
진입일 : 2019-02-01, 진입가격 : 250.768692
청산일 : 2020-04-01, 진입가격 : 250.768692, 청산가격 : 234.264252, return : 0.9342
진입일 : 2020-05-01, 진입가격 : 269.13501
청산일 : 2022-05-02, 진입가격 : 269.13501, 청산가격 : 406.066315, return : 1.5088
진입일 : 2023-04-03, 진입가격 : 409.429138
기간

In [8]:
def get_evaluation(daily_return):
    """
    cagr, dd, mdd, vol, sharpe
    투자 성과 지표
    """
    # cumulativeReturn
    cumulativeReturn = daily_return.cumprod()
    # cagr
    cagr = cumulativeReturn.iloc[-1] ** (252/len(cumulativeReturn))
    # mdd
    dd = (cumulativeReturn.cummax() - cumulativeReturn) / cumulativeReturn.cummax() * 100
    mdd= dd.max()
    vol = np.std(daily_return-1) * np.sqrt(252)
    sharpe = np.mean(daily_return-1) * 252 / vol

    print(f"기간: {daily_return.index[0].strftime('%Y/%m/%d')} ~ {daily_return.index[-1].strftime('%Y/%m/%d')}")
    print(f"최종 수익률: {cumulativeReturn.iloc[-1]}\ncagr: {cagr}\nmdd: {mdd}\nvol: {vol}\nsharpe: {sharpe}")

    return cagr, dd, mdd, vol, sharpe

cagr, _, mdd, vol, sharp = get_evaluation(book.loc[:,'return'])

기간: 2008/01/02 ~ 2023/06/30
최종 수익률: 3.1385189691944975
cagr: 1.0766829447105075
mdd: 33.7172669509494
vol: 0.14935515323254253
sharpe: 0.5698546134150987


### 4.3.2 듀얼 모멘텀 전략 구현을 위한 상대 모멘텀 전략

In [9]:
# s&p500,nasdaq,개발도상국,미국장기채,골드
stock_list = 'SPY, QQQ, TLT, GLD'
base_df = fdr.DataReader(stock_list, start='2007')
print(base_df.isna().sum(axis=0))

SPY    0
QQQ    0
TLT    0
GLD    0
dtype: int64


In [10]:
stock_df = base_df.copy()
stock_df['CASH'] = 1
print(stock_df.head())

                   SPY        QQQ        TLT        GLD  CASH
Date                                                         
2007-01-03  141.369995  43.240002  89.059998  62.279999     1
2007-01-04  141.669998  44.060001  89.599998  61.650002     1
2007-01-05  140.539993  43.849998  89.209999  60.169998     1
2007-01-08  141.190002  43.880001  89.370003  60.480000     1
2007-01-09  141.070007  44.099998  89.370003  60.849998     1


In [11]:
# 월말 데이터 추출
stock_df['STD_YM'] = stock_df.apply(lambda x: x.name.strftime('%Y-%m'), axis=1)
ym_keys = list(stock_df['STD_YM'].unique())
month_last_df = stock_df.drop_duplicates(['STD_YM'], keep="last").loc[:, ~stock_df.columns.isin(['STD_YM'])].copy()
print(month_last_df.head())

                   SPY        QQQ        TLT        GLD  CASH
Date                                                         
2007-01-31  143.750000  44.070000  87.550003  64.830002     1
2007-02-28  140.929993  43.330002  90.150002  66.480003     1
2007-03-30  142.000000  43.529999  88.279999  65.739998     1
2007-04-30  148.289993  45.959999  88.750000  67.089996     1
2007-05-31  153.320007  47.410000  86.379997  65.540001     1


In [12]:
stock_df = stock_df.reset_index()
melt_stock_df = stock_df.melt(id_vars=['Date', 'STD_YM'], var_name='CODE', value_name='Close')
print(melt_stock_df.head(5))
print(melt_stock_df.tail(5))

        Date   STD_YM CODE       Close
0 2007-01-03  2007-01  SPY  141.369995
1 2007-01-04  2007-01  SPY  141.669998
2 2007-01-05  2007-01  SPY  140.539993
3 2007-01-08  2007-01  SPY  141.190002
4 2007-01-09  2007-01  SPY  141.070007
            Date   STD_YM  CODE  Close
20755 2023-06-26  2023-06  CASH    1.0
20756 2023-06-27  2023-06  CASH    1.0
20757 2023-06-28  2023-06  CASH    1.0
20758 2023-06-29  2023-06  CASH    1.0
20759 2023-06-30  2023-06  CASH    1.0


In [13]:
month_last_df = month_last_df.reset_index()
melt_month_df = month_last_df.melt(id_vars=['Date'], var_name='CODE', value_name='Close')
print(melt_month_df.head(5))
print(melt_month_df.tail(5))

        Date CODE       Close
0 2007-01-31  SPY  143.750000
1 2007-02-28  SPY  140.929993
2 2007-03-30  SPY  142.000000
3 2007-04-30  SPY  148.289993
4 2007-05-31  SPY  153.320007
          Date  CODE  Close
985 2023-02-28  CASH    1.0
986 2023-03-31  CASH    1.0
987 2023-04-28  CASH    1.0
988 2023-05-31  CASH    1.0
989 2023-06-30  CASH    1.0


In [14]:
# print(melt_month_df.loc[melt_month_df['CODE']=='SPY'])
# for ticker in ['SPY', 'QQQ', 'TLT', 'GLD', 'CASH']:
#     ticker_df = melt_month_df.loc[melt_month_df['CODE']==ticker]
#     ret = ticker_df['Close'] / ticker_df['Close'].shift()
#     print(ticker_df.index[0], ticker_df.index[-1])
#     melt_month_df[ticker_df.index[0]:ticker_df.index[-1], ['1M_RET']] = ret
#     # print('-----------------')
# print(melt_month_df.info())
# print(melt_month_df.isin([np.NAN]).sum(axis=0))
# print(melt_month_df.head(20))

## 4.4 가치 투자 퀀트 전략

## 4.5 마치며